In [29]:
# MEL architecture
alpha = 0.33
d = [8, 16, 24, 32, 64,128,256,512] 
def create_model(input_shape, d):
    model = Sequential()
    for k in range(4):
        if( k==0 ):
            model.add(Conv2D(d[0], (3, 3), input_shape=input_shape,padding='same'))

        else:
            model.add(Conv2D(d[2*k], (3, 3),padding='same'))
        model.add(Conv2D(d[2*k+1],(3,3),padding='same'))
        model.add(BatchNormalization(momentum=0.99))
        model.add(LeakyReLU(alpha=alpha))
        model.add(MaxPooling2D(pool_size=(3, 3),padding='same'))
        model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.33))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(11, activation='softmax'))
    return model

In [21]:
# MODGD architecture 
f = [32, 64, 128, 256] 
def create_model(input_shape, f):
    model = Sequential()
    for i in range(len(f)):
        if(i==0):
            model.add(Conv2D(f[i], (3, 3), input_shape=input_shape,padding='same' ))
            
        else:
            model.add(Conv2D(f[i], (3, 3), padding='same'))  
        model.add(ReLU())
        model.add(BatchNormalization(momentum=0.99))
        model.add(MaxPooling2D(pool_size=(3, 3),padding='same'))
        model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Dropout(0.5))
    model.add(Dense(11, activation='softmax'))
    return model

In [51]:
# MODGDGRAM CRNN--------------------------------------------------------(ISMIR )
j = [32, 64, 128] 
def ReshapeLayer(x):
    
    shape = x.shape
    
    # 1 possibility: H,W*channel
    #reshape = Reshape((shape[1],shape[2]*shape[3]))(x)
    
    # 2 possibility: W,H*channel
    transpose = Permute((2,1,3))(x)
    reshape = Reshape((shape[1],shape[2]*shape[3]))(transpose)
    
    return reshape


def right_cmmr_crnn(input_shape, j):
    model = Sequential()
    for k in range(3):
        if(k == 0):
            model.add(Conv2D(j[0], (3, 3), input_shape=input_shape, padding='same'))
        else:
            model.add(Conv2D(j[ k], (3, 3), padding='same'))
        model.add(ReLU())
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(Lambda(ReshapeLayer))
    model.add(Bidirectional(GRU(32, return_sequences=True)))
    model.add(Bidirectional(GRU(32)))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(11, activation='softmax'))
    return model

In [12]:
#SIMPLE
# SIMPLE MEL CRNN (ISMIR)
def ReshapeLayer(x):
    
    shape = x.shape
    # 1 possibility: H,W*channel
    #reshape = Reshape((shape[1],shape[2]*shape[3]))(x)
    
    # 2 possibility: W,H*channel
    transpose = Permute((2,1,3))(x)
    reshape = Reshape((shape[1],shape[2]*shape[3]))(transpose)
    
    return reshape



def left_cmmr_crnn_simple(input_shape, h):
    model = Sequential()
    
    #Convolutional block
    model.add(Conv2D(12, (3, 3), input_shape=input_shape, padding='same'))

    #Max Pooling
    model.add( MaxPooling2D(pool_size=(2, 2), strides=(2, 2))) 
    
    #Reccurent block
    model.add(Lambda(ReshapeLayer))
    model.add(Bidirectional(GRU(32, return_sequences=True)))
    model.add(Bidirectional(GRU(32)))

    model.add(Flatten()) 
    model.add(Dense(512))
    model.add(Dense(11, activation='softmax'))

    return model




#  (CRNN) ( ISMIR MEL)
h = [8, 16, 32, 64, 128, 256] 

def left_cmmr_crnn(input_shape, h):
    model = Sequential()
    for k in range(3):
        if(k == 0):
            model.add(Conv2D(h[0], (3, 3), input_shape=input_shape, padding='same'))
        else:
            model.add(Conv2D(h[2 * k], (3, 3), padding='same'))
        model.add(Conv2D(h[2 * k + 1], (3, 3), padding='same'))
        model.add(LeakyReLU(alpha=alpha))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(Lambda(ReshapeLayer))
    model.add(Bidirectional(GRU(32, return_sequences=True)))
    model.add(Bidirectional(GRU(32)))
    model.add(Flatten()) 
    model.add(Dense(512))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(11, activation='softmax'))

    return model

In [12]:
#ATTENTION-----

h = [8, 16, 32, 64, 128, 256] 

def attention_left_cmmr_crnn(input_shape, h):
    inputs = Input(shape=input_shape)
    x = inputs
    for k in range(2):
        if(k == 0):
            x = Conv2D(h[0], (3, 3), padding='same')(x)
        else:
            x = Conv2D(h[2 * k], (3, 3), padding='same')(x)
        x = Conv2D(h[2 * k + 1], (3, 3), padding='same')(x)
        x = LeakyReLU(alpha=alpha)(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)

    x = Lambda(ReshapeLayer)(x)
    x = Bidirectional(GRU(32, return_sequences=True))(x)
    x = Bidirectional(GRU(32, return_sequences=True))(x)
    
    # Split output of last GRU layer into two tensors
    #query, value = tf.split(x, num_or_size_splits=2, axis=2)
    attention_output = Attention()([x, x])
    
    x = Flatten()(attention_output)
    x = Dense(256)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(11, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=x)
    return model


In [ ]:
# SWIN TRANSFORMER

input_size = (128, 128, 1) # The image size of the MNIST
patch_size = (4, 4) # Segment 28-by-28 frames into 2-by-2 sized patches, patch contents and positions are embedded
n_labels = 11 # MNIST labels


mlp_drop_rate = 0.01 # Droupout after each MLP layer
attn_drop_rate = 0.01 # Dropout after Swin-Attention
proj_drop_rate = 0.01 # Dropout at the end of each Swin-Attention block, i.e., after linear projections
drop_path_rate = 0.01 # Drop-path within skip-connections

# Self-attention parameters 
# (Fixed for all the blocks in this configuration, but can vary per block in larger architectures)
num_heads = 8 # Number of attention heads
embed_dim = 96 # Number of embedded dimensions
num_mlp = 256 # Number of MLP nodes
qkv_bias = True # Convert embedded patches to query, key, and values with a learnable additive value
qk_scale = None # None: Re-scale query based on embed dimensions per attention head # Float for user specified scaling factor

# Shift-window parameters
window_size = 4 # Size of attention window (height = width)
shift_size = window_size // 2 # Size of shifting (shift_size < window_size)

num_patch_x = input_size[0]//patch_size[0]
num_patch_y = input_size[1]//patch_size[1]


learning_rate = 0.001
weight_decay = 0.0001


with tf.device(device_name):
    # The input section
    IN = Input(input_size)
    X = IN

    # Extract patches from the input tensor
    X = transformer_layers.patch_extract(patch_size)(X)

    # Embed patches to tokens
    X = transformer_layers.patch_embedding(num_patch_x*num_patch_y, embed_dim)(X)
    # -------------------- Swin transformers -------------------- #
    # Stage 1: window-attention + Swin-attention + patch-merging

    for i in range(2):
        
        if i % 2 == 0:
            shift_size_temp = 0
        else:
            shift_size_temp = shift_size

        X = swin_layers.SwinTransformerBlock(dim=embed_dim, num_patch=(num_patch_x, num_patch_y), num_heads=num_heads, 
                                window_size=window_size, shift_size=shift_size_temp, num_mlp=num_mlp, qkv_bias=qkv_bias, qk_scale=qk_scale,
                                mlp_drop=mlp_drop_rate, attn_drop=attn_drop_rate, proj_drop=proj_drop_rate, drop_path_prob=drop_path_rate, 
                                name='swin_block{}'.format(i))(X)
    # Patch-merging
    # Pooling patch sequences. Half the number of patches (skip every two patches) and double the embedded dimensions
    X = transformer_layers.patch_merging((num_patch_x, num_patch_y), embed_dim=embed_dim, name='down{}'.format(i))(X)

    # ----------------------------------------------------------- #

    # Convert embedded tokens (2D) to vectors (1D)
    X = GlobalAveragePooling1D()(X)

    # The output section
    OUT = Dense(n_labels, activation='softmax')(X)
    
    model = keras.models.Model(inputs=[IN,], outputs=[OUT,])

    optimizer = tfa.optimizers.AdamW(
         learning_rate=learning_rate, weight_decay=weight_decay
     )

    model.compile(
         optimizer=optimizer,
         loss=keras.losses.CategoricalCrossentropy(from_logits=True),
         metrics=[
             keras.metrics.CategoricalAccuracy(name="accuracy"),
             keras.metrics.TopKCategoricalAccuracy(5, name="top-5-accuracy"),
         ],
    )
    early_stop = EarlyStopping(monitor='val_loss', mode='min', patience=2)
    checkpoint_filepath = ""
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
         checkpoint_filepath,
         monitor="accuracy",
         save_best_only=True,
         save_weights_only=True,
    )

    history = model.fit(
         x=x_train,
         y=y_train,
         batch_size=32,
         validation_data=(x_test, y_test),
         epochs=100,
         callbacks=[checkpoint_callback,early_stop],
    )

    model.load_weights(checkpoint_filepath)
    _, accuracy, top_5_accuracy = model.evaluate(x_test, y_test)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")
    #Compile the model
    opt = keras.optimizers.Adam(learning_rate=1e-3, clipvalue=0.5)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy',])


In [ ]:
# VISION TRANSFORMER


learning_rate = 0.001
weight_decay = 0.0001
batch_size = 256
num_epochs = 100
image_size = 72  # We'll resize input images to this size
patch_size = 6  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 8
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 16
mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier


data_augmentation = keras.Sequential(
    [
       layers.Normalization(),
        layers.Resizing(image_size, image_size),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(
            height_factor=0.1, width_factor=0.1
        ),
    ],
    name="data_augmentation",
)
# Compute the mean and the variance of the training data for normalization.
data_augmentation.layers[0].adapt(x_train)



def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x



class Patches(layers.Layer):
    def __init__(self, patch_size):
        super().__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches
    
    
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super().__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded
    
    
    
    
def create_vit_classifier():
    inputs = layers.Input(shape=input_shape)
    # Augment data.
    augmented = data_augmentation(inputs)
    # Create patches.
    patches = Patches(patch_size)(augmented)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    # Classify outputs.
    logits = layers.Dense(num_classes)(features)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model


def run_experiment(model):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.CategoricalAccuracy(name="accuracy"),
            keras.metrics.TopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ],
    )

    checkpoint_filepath = ""
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(
        x=x_train,
        y=y_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_data=(x_test,y_test),
        callbacks=[checkpoint_callback],
    )

    model.load_weights(checkpoint_filepath)
    _, accuracy, top_5_accuracy = model.evaluate(x_test, y_test)
    #print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    #print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    return history
   
with tf.device(device_name):
    vit_classifier = create_vit_classifier()
    history = run_experiment(vit_classifier)